In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, transpile, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import IQP
from qiskit.circuit import ParameterVector

import pennylane as qml

from src import QCIBM_utilities as ut

In [2]:
nq = 2
alpha_init = np.array([[1,0],[0.1,0.3]])
Gamma_init = np.array([1,1])
Delta_init = np.array([0.5, 0.2])
Sigma_init = np.array([0,0.1])

In [3]:
backend = Aer.get_backend('qasm_simulator')

In [4]:
theta = ParameterVector('θ', int(nq*(nq+7)/2))
alpha, Gamma, Delta, Sigma = ut.decompose_params(theta, nq)
qc = ut.QCIBM(alpha, Gamma, Delta, Sigma, nq)
qc = transpile(qc, backend)

#x1,p1 = ut.sample_QCIBM(theta_bind, nq, backend, nsamples = 32)
#x2, p2 = ut.sample_QCIBM(theta_bind, nq, backend, 32, qc.bind_parameters({theta: ut.compose_params(alpha, Gamma, Delta, Sigma)}))



In [5]:
# Samples modes
T = 3
ps = 0.4
pi_modes = np.zeros((T,nq)) 
for t in range(T):
    for i in range(nq):
        pi_modes[t,i] = np.random.binomial(1, ps)
print('modes = ', pi_modes)

modes =  [[1. 0.]
 [0. 1.]
 [0. 1.]]


In [6]:
# Sample from pi
p_pi = 0.6
n_train = 512
sampleshots = 512

In [7]:
kernel_params = [0.1,1,10]

In [8]:
theta = ParameterVector('θ', int(nq*(nq+7)/2))
alpha, Gamma, Delta, Sigma = ut.decompose_params(theta, nq)
qc = ut.QCIBM(alpha, Gamma, Delta, Sigma, nq)
qc = transpile(qc, backend)

alpha_init = np.array([[1,0],[0.1,0.3]])
Gamma_init = np.array([1,1])
Delta_init = np.array([0.5,0.2])
Sigma_init = np.array([0,0.1])
theta_init = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)

xsamples, phat = ut.sample_circuit( backend, sampleshots, qc.bind_parameters({theta: theta_init}))
ysamples, pi_vec = ut.sample_target_pdf(n_train, pi_modes, p_pi)
Lmmd = ut.KL_Gauss_Loss(xsamples, ysamples, kernel_params)
print('Lmmd = ', Lmmd)

h = 0.01
nablaL = ut.KL_Gauss_grad(qc, theta, theta_init, xsamples, ysamples, backend, kernel_params)

for k in range(len(theta)):
    theta_plus = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)
    theta_plus[k] += h
    xsamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_plus}))
    Lmmd_plus = ut.KL_Gauss_Loss(xsamples, ysamples, kernel_params)
    theta_minus = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)
    theta_minus[k] -= h
    xxsamples, pphat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_minus}))
    Lmmd_minus = ut.KL_Gauss_Loss(xxsamples, ysamples, kernel_params)
     
    num_grad = (Lmmd_plus - Lmmd_minus)/(2*h)
    print('num grad[',k,'] = ', num_grad)
    print('anal grad[',k,'] = ', nablaL[k])
    print('Δ[',k,'] = ', np.abs(num_grad - nablaL[k]))


Lmmd =  0.06111530846336821


NameError: name 'Ksamples' is not defined

In [ ]:
P = np.zeros((500,500))
P[1,1] = 1
Q = np.zeros((500,500))
Q[1,1] = 1
print(ut.entropy_relative(P,Q))

In [ ]:
xsamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_init}))
Sigma = ut.estimate_covariance_embedding(xsamples, kernel_params )

h = 0.01

for k in range(len(theta)):
    
    theta_plus = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)
    theta_plus[k] += h
    xsamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_plus}))
    Sigmaplus = ut.estimate_covariance_embedding(xsamples, kernel_params)
    
    theta_minus = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)
    theta_minus[k] -= h
    ysamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_minus}))
    Sigmaminus = ut.estimate_covariance_embedding(ysamples, kernel_params)
    
    numderiv = (Sigmaplus - Sigmaminus)/(2*h)
    
    theta_plus = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)
    theta_plus[k] += 0.5*np.pi
    xsamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_plus}))
    Sigmaplus = ut.estimate_covariance_embedding(xsamples, kernel_params)
    
    theta_minus = ut.compose_params(alpha_init, Gamma_init, Delta_init, Sigma_init)
    theta_minus[k] -= 0.5*np.pi
    ysamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_minus}))
    Sigmaminus = ut.estimate_covariance_embedding(ysamples, kernel_params)
    
    analderiv = (Sigmaplus - Sigmaminus)/2
    
    print('num grad[',k,'] = ')
    print(numderiv)
    print('anal grad[',k,'] = ')
    print(analderiv)
    print('Δ[',k,'] = ', np.linalg.norm(analderiv - numderiv))


In [ ]:
xsamples, phat = ut.sample_circuit(backend, sampleshots, qc.bind_parameters({theta: theta_init}))
ysamples, pi_vec = ut.sample_target_pdf(n_train, pi_modes, p_pi)
Sigma = ut.estimate_covariance_embedding(xsamples, kernel_params )
Targ = ut.estimate_covariance_embedding(ysamples, kernel_params )

h = 0.001

num_grad = np.zeros(Sigma.shape)
    
for i in range(Sigma.shape[0]):
    for j in range(Sigma.shape[1]):
        Sigmaplus = np.copy(Sigma)
        Sigmaplus[i,j] += h
        Sigmaminus = np.copy(Sigma)
        Sigmaminus[i,j] -= h
        
        
            
        DKLp = ut.entropy_relative(Sigmaplus, Targ)
        DKLm = ut.entropy_relative(Sigmaminus, Targ)
        #DKLp = qml.math.relative_entropy(Sigmaplus, Targ)
        #DKLm = qml.math.relative_entropy(Sigmaminus, Targ)

        
        num_grad[i,j] = (DKLp - DKLm)/(2*h)
    

LP = ut.logm(Sigma)
LQ = ut.logm(Targ)
anal_grad = LP + LP.T - np.multiply(np.eye(LP.shape[0]),LP) - (  LQ + LQ.T - np.multiply(np.eye(LQ.shape[0]),LQ) )
anal_grad_noob = LP - LQ
 
    
print('num grad = ')
print(num_grad)
print('anal grad = ')
print(anal_grad)
print('anal grad noob = ')
print(anal_grad_noob)
print('Δ = ', np.linalg.norm(anal_grad - num_grad))


In [ ]:
def estimate_covariance_embedding_projection(xsamples,samples, Ksqrt, ker_params):

    Sigmap = np.zeros((samples.shape[0],samples.shape[0]))

    for i in range(xsamples.shape[0]):
        ki = ut.compute_kernel_vector(samples,xsamples[i], ker_params)
        Sigmap += np.outer(ki,ki)

    delta = 10**(-10)
    Sigmap = 0.5*(Sigmap + Sigmap.conj().T)
    print(Sigmap)
    eigval, eigvec = np.linalg.eigh(Sigmap)
    print('Eigvals(Sigma) = ', eigval)
    Sigmap = 1/(2*xsamples.shape[0]) * Ksqrt @ (Sigmap + Sigmap.T + delta*np.eye(Sigmap.shape[0])) @ Ksqrt


    return(Sigmap)

In [ ]:
rng = np.random.default_rng(27031995)
proj_nsamples = 512
nq = xsamples.shape[1]
samples = np.zeros((proj_nsamples,nq))
for i in range(nq):
    samples[:,i] =  rng.integers(0, 2, proj_nsamples)
K = ut.compute_graham_gauss(samples, kernel_params)
delta = 10**(-8)
Ksqrt = np.linalg.inv(ut.sqrtm(K + delta * np.eye(K.shape[0])))

rho = ut.estimate_covariance_embedding_projection(xsamples,samples, Ksqrt, kernel_params)
sigma= ut.estimate_covariance_embedding_projection(ysamples,samples, Ksqrt, kernel_params)

In [ ]:
rvals, rvecs = np.linalg.eig(rho)
rvals = np.real(rvals)
svals, svecs = np.linalg.eig(sigma)
svals = np.real(svals)

print('eig(rho) = ',rvals)
print('eig(sigma) = ',svals)

tol = 10**(-5)

print('non-zeros eig = ',rvals[rvals>tol])

print('res = ', rho@svecs[:,svals<tol])

if any(rvals<tol):
    T = sigma@rvecs[:,rvals>tol]
    flags = []
    [flags.append(np.linalg.norm(T[:,i]) < tol) for i in range(np.sum(rvals>tol))]
    print(flags)
    if any(flags):
        print(9999)

H = -rvals[rvals > tol]@np.log(rvals[rvals > tol])
rvals[rvals < tol] = 0
svals[svals < tol] = 1

IP = abs(np.conj(svecs).T@rvecs)**2
Hcross = -rvals@IP@np.log(svals)

print(np.real(Hcross - H -np.trace(rho) + np.trace(sigma)))